In [1]:
import sys
import os

# Go up one level from notebooks/ to project root
sys.path.append(os.path.abspath(".."))

from utils.data_fetcher import fetch_stock_data


In [2]:
from utils.data_fetcher import fetch_stock_data

symbols = ["TCS.NS", "INFY.NS", "ICICIBANK.NS", "RELIANCE.NS", "^NSEI"]

for symbol in symbols:
    df = fetch_stock_data(symbol)
    print(f"{symbol} shape: {df.shape}")


TCS.NS shape: (1788, 5)
INFY.NS shape: (1788, 5)
ICICIBANK.NS shape: (1788, 5)
RELIANCE.NS shape: (1788, 5)
^NSEI shape: (1783, 5)


In [3]:
df = fetch_stock_data("TCS.NS")
print(df.head())
print(df["Close"].shape)
print(type(df["Close"]))


Price             Close         High          Low         Open   Volume
Date                                                                   
2018-01-01  1154.967407  1176.446217  1150.339812  1170.989197  1351760
2018-01-02  1148.681152  1165.445186  1143.878974  1161.254151  1920290
2018-01-03  1151.933350  1165.183038  1148.680987  1149.030257  1257120
2018-01-04  1159.987305  1162.126413  1152.522062  1156.887676   913082
2018-01-05  1174.000977  1178.497581  1156.887790  1156.887790  1153706
(1788,)
<class 'pandas.core.series.Series'>


In [4]:
from utils.indicators import add_technical_indicators

# Fetch and enrich for one stock
df = fetch_stock_data("TCS.NS")

print(type(df["Close"]))
print(df["Close"].shape)

df_ind = add_technical_indicators(df)

print(df_ind.tail())


<class 'pandas.core.series.Series'>
(1788,)
Price             Close     High          Low         Open   Volume  \
Date                                                                  
2025-03-21  3578.100098  3616.25  3465.000000  3475.000000  4161925   
2025-03-24  3628.949951  3649.75  3566.600098  3600.000000  1834751   
2025-03-25  3657.750000  3710.00  3638.000000  3645.000000  3135390   
2025-03-26  3635.800049  3680.50  3625.850098  3647.750000  1734499   
2025-03-27  3651.199951  3663.00  3610.500000  3614.850098  2528474   

Price             rsi       macd  bollinger_h  bollinger_l      stoch  \
Date                                                                    
2025-03-21  39.553981  16.903192  3750.155887  3406.569137  71.094051   
2025-03-24  46.304572  23.580865  3718.200875  3423.479154  88.876979   
2025-03-25  49.728939  29.340066  3681.912185  3446.942844  78.867543   
2025-03-26  47.255396  30.916061  3673.494914  3451.380111  69.989909   
2025-03-27  49.16584

In [5]:
from utils.windowizer import create_sequences

X, y = create_sequences(df_ind, window_size=30)

print("X shape:", X.shape)  # (samples, 30, features)
print("y shape:", y.shape)  # (samples,)


X shape: (1724, 30, 10)
y shape: (1724,)


In [6]:
from utils.scaler import scale_dataframe

df_scaled, scaler = scale_dataframe(df_ind)
df_scaled.head()


Price,Close,High,Low,Open,Volume,rsi,macd,bollinger_h,bollinger_l,stoch,adx
Date,,,,,,,,,,,
2018-02-19,0.014403,0.016838,0.015478,0.019154,0.017807,0.444764,0.362460,0.033025,0.023378,0.093057,0.733679
2018-02-20,0.017261,0.021084,0.019931,0.016876,0.021450,0.480893,0.380875,0.033073,0.023225,0.170306,0.685051
2018-02-21,0.030442,0.031456,0.024309,0.023038,0.035078,0.618631,0.434187,0.032203,0.023108,0.504819,0.658849
2018-02-22,0.031509,0.033650,0.032814,0.033485,0.039322,0.628157,0.473905,0.031592,0.023029,0.531907,0.638112
2018-02-23,0.034968,0.034987,0.028674,0.033217,0.038181,0.659061,0.509670,0.029665,0.023665,0.623951,0.607693


In [8]:
from models.gru_trainer import train_gru_model
from utils.windowizer import create_sequences
from utils.scaler import scale_dataframe
from utils.indicators import add_technical_indicators
from utils.data_fetcher import fetch_stock_data

# Fetch, process, scale
df = fetch_stock_data("TCS.NS")
df = add_technical_indicators(df)
df_scaled, scaler = scale_dataframe(df)

# Create windowed sequences
X, y = create_sequences(df_scaled, window_size=30)

# Train model
model, history, (X_train, X_val, y_train, y_val) = train_gru_model(X, y)


Epoch 1/100


/Users/rakshitshah/Desktop/Portfolio_Projects/smartstock-ml/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0808 - val_loss: 0.0063
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0095 - val_loss: 0.0033
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0059 - val_loss: 0.0035
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0052 - val_loss: 9.5803e-04
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0046 - val_loss: 0.0021
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0037 - val_loss: 0.0025
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0040 - val_loss: 0.0016
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0036 - val_loss: 9.0667e-04
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0033 - val_loss: 8.8688e-04
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0034 - val_loss: 8.8671e-04
Epoch 11/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0034 - val_loss: 9.5285e-04
Epoch 12/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("models/gru_model.keras")


In [9]:
from utils.windowizer import create_sequences
window_size = 30
recent_df = df_scaled.tail(window_size + 2)  # try 32 rows just to be safe

X_pred, _ = create_sequences(recent_df, window_size=window_size)
print("X_pred shape:", X_pred.shape)



X_pred shape: (1, 30, 10)


In [10]:
scaled_prediction = model.predict(X_pred)[-1][0]  # get last prediction
print("Scaled prediction:", scaled_prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Scaled prediction: 0.74113655


In [11]:
import pandas as pd

# Copy the last row and insert predicted scaled Close
dummy_row = df_scaled.iloc[[-1]].copy()
dummy_row["Close"] = scaled_prediction  # replace only the Close value

# Inverse transform
inv_row = scaler.inverse_transform(dummy_row)

# Get predicted price using original index from df_ind
predicted_price = inv_row[0][df_ind.columns.get_loc("Close")]

print(f"📈 Predicted Close Price: ₹{predicted_price:.2f}")


📈 Predicted Close Price: ₹3633.01


In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Make predictions on validation set
y_val_pred = model.predict(X_val).flatten()

# Inverse scale both y_val and y_val_pred
# We rebuild dummy DataFrames to inverse-transform
def inverse_scale_y(y_scaled, original_df, col_name="Close"):
    dummy_df = original_df.iloc[:len(y_scaled)].copy()
    dummy_df[col_name] = y_scaled
    inv = scaler.inverse_transform(dummy_df)
    return inv[:, original_df.columns.get_loc(col_name)]

y_val_true = inverse_scale_y(y_val, df_ind)
y_val_pred_actual = inverse_scale_y(y_val_pred, df_ind)

# Evaluate
mae = mean_absolute_error(y_val_true, y_val_pred_actual)
rmse = np.sqrt(mean_squared_error(y_val_true, y_val_pred_actual))
r2 = r2_score(y_val_true, y_val_pred_actual)

print(f"📊 MAE: ₹{mae:.2f}")
print(f"📉 RMSE: ₹{rmse:.2f}")
print(f"📈 R² Score: {r2:.4f}")


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  
📊 MAE: ₹73.30
📉 RMSE: ₹93.69
📈 R² Score: 0.8910
